In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension ("tabulator", template="material", sizing_mode="stretch_width")
import holoviews as hv
import hvplot.pandas
import datetime as dt

In [2]:
# loading data and showing it as a table on top of the dashboard
#https://docs.google.com/spreadsheets/d/13ikD5WpjmapBlKY4j2dTrwsZ-du3pdfpHhjIYwugcxM/edit?usp=sharing

sheet_id="13ikD5WpjmapBlKY4j2dTrwsZ-du3pdfpHhjIYwugcxM"    
df_google_sheet= pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")    #add this /export?format=csv
useful_data=df_google_sheet
df_google_sheet=df_google_sheet.interactive()
df_google_sheet1=df_google_sheet.hvplot.table( sortable=True, selectable=True)  
#df_google_sheet1

In [3]:
# not started table change the path according to location
all_uc=pd.read_csv("/Users/smhz/Downloads/HAMZA.csv")


In [4]:
started_uc=list(useful_data["Cluster_Code"].unique())
not_started=all_uc
#started_uc
for i in started_uc:
    #print(i)
    #count[(count['A14']==1)]
    not_started=not_started[(not_started["ID"]!=i)]
not_started
#visualisation
not_started_table = pn.widgets.Tabulator(
    not_started, pagination='remote', layout='fit_columns', page_size=80, sizing_mode='stretch_width', name="Not started"
    #header_filters=filters , editors=editors
)
not_started_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'b4d9850f-5f97-4a40-a613-57803c2c9281': {'defs': …

In [5]:
# pending table
#useful_data.columns
all_started=useful_data[["Cluster_Code","DISTRICT","TEHSIL","RURAL_URBAN","RIVER_BASIN","WATER_SHED","UNION_COUNCIL","Sampled_UC_Status","A14"]]
all_started
# table 3 
#get the bifercaton of male and female from column a14 ,1 means male, 2 means female
count=all_started[["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"A14"]]
count_total=count.groupby(["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status'], as_index=False)["A14"].count()  #this gets all the names and the total surveys submitted for each cluster also as_index=False is extremely important to add
count_male=count[(count['A14']==1)].groupby(["Cluster_Code","A14"])["A14"].count()  # gets male count
#print(count_male)
count_female=count[(count['A14']==2)].groupby(["Cluster_Code","A14"])["A14"].count() # gets female count
#print(count_female)
merge=pd.merge(count_male,count_female, how="outer", on="Cluster_Code") #left_on="Cluster_Code", right_on="Cluster_Code")
#print(merge)
#print(merge.dtypes)
#merge_total=count_total.join(merge)
merge_total=pd.merge(count_total,merge, how="outer", on="Cluster_Code")
#print(count_total)
#print(count.dtypes)
# rename columns
merge_total.rename(columns={"A14":"Total", 
                            "A14_x":"Male",
                            "A14_y":"Female"},
inplace=True, errors="raise")
#changing Nan to 0
merge_total=merge_total.fillna(0)
pending=merge_total[(merge_total["Male"]<20) | (merge_total["Female"]<4)]
#pending["Female"]
pending

#count_total
#count_male
#count_female
#merge_total
#visualisation
pending_table = pn.widgets.Tabulator(
    pending, pagination='remote', layout='fit_columns', page_size=80, sizing_mode='stretch_width', name="In progress"
    #header_filters=filters , editors=editors
)
pending_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'41651c21-661e-40eb-aa1e-192a5102c44e': {'defs': …

In [6]:
#completed table
merge_total=merge_total.fillna(0)
completed=merge_total[(merge_total["Male"]>20) & (merge_total["Female"]>4)]
completed
# create tabular form
complete_table = pn.widgets.Tabulator(
    completed, pagination='remote', layout='fit_columns', page_size=80, sizing_mode='stretch_width' ,name="Completed"
    #header_filters=filters , editors=editors
)
complete_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'2130c68c-1b2f-423c-8e62-f7e5427b8d58': {'defs': …

In [7]:
#date slider for daily progress
#adding date filter
#first is to make the widget
datetime_range_slider = pn.widgets.DatetimeRangeSlider(
    name='Datetime Range Slider',
    start=dt.datetime(2022, 12, 1), end=dt.datetime(2022, 12, 31),
    value=(dt.datetime(2022, 12, 1), dt.datetime(2022, 12, 31)),
    step=10000
)

datetime_range_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'9aacb32e-04f7-4c04-ba97-a0e0fbc0db8c': {'defs': …

In [8]:
#date filter for daily progress
#make a dataframe that has filtered data
date=useful_data#[["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"A14","Survey_Day"]]
date["Survey_Day"]=pd.to_datetime(date["Survey_Day"]) #convert format from object to date_type
# filter_table2 = pn.widgets.Tabulator(
#     merge_total, pagination='remote', layout='fit_columns', page_size=10, sizing_mode='stretch_width'
#     #header_filters=filters , editors=editors
# )

In [9]:
# daily progress  
#get the bifercaton of male and female from column a14 ,1 means male, 2 means female
count=date[["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"A14","Survey_Day"]]
count_total=count.groupby(["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"Survey_Day"], as_index=False)["A14"].count()  #this gets all the names and the total surveys submitted for each cluster also as_index=False is extremely important to add
count_male=count[(count['A14']==1)].groupby(["Cluster_Code","A14"])["A14"].count()  # gets male count
#print(count_male)
count_female=count[(count['A14']==2)].groupby(["Cluster_Code","A14"])["A14"].count() # gets female count
#print(count_female)
merge=pd.merge(count_male,count_female, how="outer", on="Cluster_Code") #left_on="Cluster_Code", right_on="Cluster_Code")
#print(merge)
#print(merge.dtypes)
#merge_total=count_total.join(merge)
merge_total=pd.merge(count_total,merge, how="outer", on="Cluster_Code")
#print(count_total)
#print(count.dtypes)
# rename columns
merge_total.rename(columns={"A14":"Total", 
                            "A14_x":"Male",
                            "A14_y":"Female"},
inplace=True, errors="raise")
#changing Nan to 0
merge_total=merge_total.fillna(0)
merge_total
#count_total
#count_male
#count_female



,Cluster_Code,DISTRICT,TEHSIL,RIVER_BASIN,WATER_SHED,UNION_COUNCIL,Sampled_UC_Status,Survey_Day,Total,Male,Female
0,2,Chagai,Dalbandin,Hamun-e-Mushkhel,Gowanakkoh,CHALGAZI,Actual,2022-12-04,1,11,8.0
1,2,Chagai,Dalbandin,Hamun-e-Mushkhel,Gowanakkoh,CHALGAZI,Actual,2022-12-05,18,11,8.0
2,28,Khuzdar,Khuzdar,Hingol,Toba,TOTAK,Actual,2022-12-02,10,9,1.0
3,46,Harnai,Harnai,Nari,Harnai,NASAK,Actual,2022-12-05,11,11,0.0
4,56,Ziarat,Sinjawi,Nari,Chinjan,POI,Actual,2022-12-03,24,18,6.0
5,57,Ziarat,Ziarat,Nari,Shahrag,ZIARAT,Actual,2022-12-01,1,16,9.0
6,57,Ziarat,Ziarat,Nari,Shahrag,ZIARAT,Actual,2022-12-02,24,16,9.0
7,58,Ziarat,Ziarat,Nari,Shahrag,KWAS,Actual,2022-12-01,23,16,7.0
8,61,Killa Abdullah,Killa Abdullah,Pishin,Killa Abdullah,ARAMBAI,Actual,2022-12-01,13,24,0.0
9,61,Killa Abdullah,Killa Abdullah,Pishin,Killa Abdullah,ARAMBAI,Actual,2022-12-02,11,24,0.0


In [18]:
#visualisation of daily progress
filter_table2 = pn.widgets.Tabulator(
    merge_total, pagination='remote', layout='fit_columns', page_size=80, sizing_mode='stretch_width', name="Daily progress"
    #header_filters=filters , editors=editors
)
filter_table2.add_filter(datetime_range_slider,"Survey_Day")    # first input is the widget and the second input is the column on which the filter is added

tab_daily_progress=pn.Column(datetime_range_slider,filter_table2, name="Daily progress")


In [23]:
tabulator_editors = {
    'Cluster_Code': {'type': 'number', 'max': 80, 'step': 1},
    # 'bool': {'type': 'tickCross', 'tristate': True, 'indeterminateValue': None},
    # 'str': {'type': 'list', 'valuesLookup': True}
}

# header_filter_table = pn.widgets.Tabulator(
#     df[['float', 'bool', 'str']], height=140, width=400, layout='fit_columns',
#     editors=tabulator_editors, header_filters=True
# )
# header_filter_table

In [27]:
#another table for cluster search progress
filter_table3 = pn.widgets.Tabulator(
    merge_total, pagination='remote', layout='fit_columns', page_size=80, sizing_mode='stretch_width', name="Daily progress",
    header_filters=tabulator_editors  
)

In [32]:
#making a tab for all this data
tab_daily_progress=pn.Column(datetime_range_slider,filter_table2,"UC Code wise search",filter_table3, name="Daily progress")


In [33]:
#creating dashboard
tabs=pn.Tabs(not_started_table,pending_table,complete_table,tab_daily_progress, dynamic=True)
tabs.show()

Launching server at http://localhost:63832


2022-12-06 12:50:08,950 ERROR: panel.reactive - Callback failed for object named "Daily progress" changing property {'filters': []} 
Traceback (most recent call last):
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/panel/reactive.py", line 319, in _process_events
    self.param.update(**self_events)
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/param/parameterized.py", line 1898, in update
    self_._batch_call_watchers()
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/param/parameterized.py", line 2059, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/param/parameterized.py", line 2021, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/panel/widgets/tables.py", line 1460, in _update_cds
    self._update_style(recompute)
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/panel

Task exception was never retrieved
future: <Task finished name='Task-170440' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at /Users/smhz/Library/Python/3.8/lib/python/site-packages/tornado/websocket.py:1090> exception=WebSocketClosedError()>
Traceback (most recent call last):
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/tornado/websocket.py", line 1092, in wrapper
    await fut
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/smhz/Library/Python/3.8/lib/python/site-packages/tornado/websocket.py", line 1094, in wrapper
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x107282160>>, <Task finished name='Task-170427' coro=<ServerSession